In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VineAnalysis").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Music_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [ ]:
clean_df = df.dropna()
clean_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [ ]:
# Create the vine DataFrame
vine_df = clean_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3LI5TRP3YIDQL|          5|            0|          0|   N|                Y|
|R3LGC3EKEG84PX|          5|            0|          0|   N|                Y|
| R9PYL3OYH55QY|          5|            0|          1|   N|                Y|
|R3PWBAWUS4NT0Q|          3|            0|          0|   N|                Y|
|R15LYP3O51UU9E|          5|            0|          0|   N|                Y|
|R1AD7L0CC3DSRI|          5|            0|          0|   N|                Y|
|R32FE8Y45QV434|          5|            0|          0|   N|                Y|
|R3NM4MZ4XWL43Q|          5|            1|          2|   N|                Y|
|R3H4FXX6Q7I37D|          4|            0|          0|   N|                Y|
|R30L5PET7LFFDC|          5|            1|          1|   N|     

**Determine Bias of Vine Reviews**

In [ ]:
# Filter data and create DataFrame to retrieve all rows where the total_votes count is equal to or greater than 20
from pyspark.sql.functions import col
totalVotes_df = vine_df.filter(col("total_votes") >= 20)
totalVotes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N|
| RFN4PNRUD1UW2|          4|           21|         22|   N|                N|
| RO7EBJEP7IHIX|          5|           26|         26|   N|                N|
|R1CVS4MK9RTNNP|          2|           11|         22|   N|     

In [ ]:
# Filter totalVotes DataFrame and create new DataFrame to retrieve all rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
percentVotes_df = totalVotes_df.withColumn("percent_votes", col("helpful_votes")/col("total_votes")).alias("percent_votes>0.5")
percentVotes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|      percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y| 0.9615384615384616|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N| 0.9615384615384616|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|               0.95|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y| 0.9047619047619048|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N| 0.9583333333333334|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|                1.0|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N| 0.9733333333333334|
| RFN4PNRUD1UW2|    

In [ ]:
# Create a DataFrame that retrieves all the rows where a review was written as part of the Vine program (paid)
paid_df = percentVotes_df.filter(col("vine")== "Y")
paid_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| R84VUCDBCI29U|          3|           18|         28|   Y|                N|0.6428571428571429|
| R4V3ICFDTIDIF|          4|           20|         21|   Y|                N|0.9523809523809523|
| R7ZY9DI5Z6C6Z|          2|            6|         27|   Y|                N|0.2222222222222222|
|R1JZ0JAPW83WFS|          4|           54|         58|   Y|                N|0.9310344827586207|
| RSPA592OKHQMZ|          2|           28|         78|   Y|                N| 0.358974358974359|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
only showing top 5 rows



In [ ]:
unpaid_df = percentVotes_df.filter(col("vine")== "N")
unpaid_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|0.9615384615384616|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|0.9615384615384616|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|              0.95|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|0.9047619047619048|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|0.9583333333333334|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
only showing top 5 rows



In [ ]:
# Total number of reviews, total number of 5 star views, percentage of 5 stars reviews by type of review
totalReviews_df = percentVotes_df.groupBy("vine").agg(
    count(col("vine")).alias("total_reviews"),
    count(when(col("star_rating") == 5, True)).alias("total_5_star_reviews"),
    (count(when(col("star_rating") == 5, True))/count(col("vine"))*100).alias("%_5_star_reviews"))

totalReviews_df.show()

+----+-------------+--------------------+-----------------+
|vine|total_reviews|total_5_star_reviews| %_5_star_reviews|
+----+-------------+--------------------+-----------------+
|   Y|           13|                   0|              0.0|
|   N|       153124|               70354|45.94576944175962|
+----+-------------+--------------------+-----------------+

